|<h2>Book:</h2>|<h1><a href="https://open.substack.com/pub/mikexcohen/p/llm-breakdown-16-tokenization-words" target="_blank">50 ML projects to understand LLMs</a></h1>|
|-|:-:|
|<h2>Project:</h2>|<h1><b>[19] LLM loss function</b></h1>|
|<h2>Author:<h2>|<h1>Mike X Cohen, <a href="https://sincxpress.com" target="_blank">sincxpress.com</a></h1>|

<br>

<i>Using the code without reading the book may lead to confusion or errors.</i>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
### matplotlib adjustments (commented lines are for dark mode)

# svg plots (higher-res)
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

plt.rcParams.update({
    # 'figure.facecolor': '#282a2c',
    # 'figure.edgecolor': '#282a2c',
    # 'axes.facecolor':   '#282a2c',
    # 'axes.edgecolor':   '#DDE2F4',
    # 'axes.labelcolor':  '#DDE2F4',
    # 'xtick.color':      '#DDE2F4',
    # 'ytick.color':      '#DDE2F4',
    # 'text.color':       '#DDE2F4',
    'axes.spines.right': False,
    'axes.spines.top':   False,
    'axes.titleweight': 'bold',
    'axes.labelweight': 'bold',
    'savefig.dpi':300,
})

# **Part 1: Manual negative log likelihood loss**

In [ ]:
# start with three outputs (raw model outputs for three tokens in the vocab)
model_output = np.array([ -1, 2.3, .1 ])
print('Raw model outputs:\n  ',model_output,'\n')

# NLLLoss expects log-softmax inputs!
softmax = np.exp(model_output) / np.exp(model_output).sum()
logsoftmax_output = np.log(softmax)
print('Log-softmax model outputs:\n  ',logsoftmax_output,'\n')


# check the loss for different targets
for target in range(len(model_output)):

  # calculate the loss
  theloss = -logsoftmax_output[target]

  # and print
  print(f'Loss is {theloss:.4f} when correct output is index "{target}"')

# **Part 2: NLLLoss in PyTorch**

In [ ]:
# create a loss function instance
loss_function = nn.NLLLoss()
dir(loss_function)

In [ ]:
# start with three outputs (raw model outputs for three tokens in the vocab)
model_output = torch.tensor([[ -1, 2.3, .1 ]])
print('Raw model outputs:\n  ',model_output[0].tolist(),'\n')

# NLLLoss expects log-softmax inputs!
logsoftmax_output = F.log_softmax(model_output,dim=-1)
print('Log-softmax model outputs:\n  ',logsoftmax_output[0],'\n')


# check the loss for different targets
for target in range(len(model_output[0])):

  # which output is the target (correct response)?
  target = torch.tensor([target])

  # calculate the loss
  theloss = loss_function(logsoftmax_output,target)

  # and print
  print(f'Loss is {theloss.item():.4f} when correct output is index "{target.item()}"')

In [ ]:
# create some tensors
T1 = torch.tensor([3.4])
T2 = torch.tensor([3,4])
T3 = torch.tensor([ [3.1],[4] ])

print(f'T1 is of type {type(T1)}, and T1.item() is of type {type(T1.item())}\n')
# print(f'T2 is of type {type(T2)}, and T2.item() is of type {type(T2.item())}\n')
# print(f'T3 is of type {type(T3)}, and T3.item() is of type {type(T3.item())}\n')

In [ ]:
# otherwise:
T3.numpy()

# **Part 3: Cross-entropy loss for next-token prediction**

In [ ]:
# load pretrained GPT-2 model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained('gpt2')
model.eval()

In [ ]:
# from the Beatles :<)
text = 'There are places I remember all my life, though some have changed. Some forever, not for better, some have gone and some remain.'

tokens = tokenizer.encode(text,return_tensors='pt')
print(f'There are {len(tokens[0])} tokens, {len(set(tokens[0]))} of which are unique.')

In [ ]:
# targets are the next-tokens
targets = tokens[0,1:]

# print the input-target table
print(' Input | Target')
print('-------+--------')
for i in range(len(tokens[0])-1):
  print(f' {tokens[0,i]:4}  |  {targets[i]:4}')

In [ ]:
# forward pass and get logits
with torch.no_grad():
  outputs = model(tokens)
logits = outputs.logits[0,:-1,:]

# NLLLoss expects log-softmax
logits_log_sm = F.log_softmax(logits,dim=-1)

print(f'Shape of logits: {list(logits.shape)}')
print(f'Shape of targets: {list(targets.shape)}')

In [ ]:
# losses
loss = loss_function(logits_log_sm,targets)
loss

# **Part 4: Manual loss calculation**

In [ ]:
losses = np.zeros(len(tokens[0])-1)

for i in range(len(tokens[0])-1):

  # get the log-softmax for this token
  log_sm = F.log_softmax( outputs.logits[0,i,:] ,dim=-1)

  # pick out the logsm for target
  losses[i] = -log_sm[tokens[0,i+1]].item()

losses.mean()

In [ ]:
plt.figure(figsize=(12,4))

loss_scale = (losses-losses.min()) / (losses.max()-losses.min())
xlabels = [tokenizer.decode(t) for t in tokens[0][1:]]

plt.bar(range(len(losses)),losses,edgecolor='k',linewidth=.3,color=plt.cm.magma(loss_scale))
plt.gca().set(xticks=range(len(losses)),xlabel='Token',ylabel='Loss',
              xticklabels=xlabels)
plt.xticks(rotation=90)

plt.tight_layout()
plt.savefig('ch4_proj19_part4.png')
plt.show()

# **Part 5: Using the model's internal loss calculation**

In [ ]:
with torch.no_grad():
  outputs = model(tokens,labels=tokens)
outputs.loss

# **Part 6: Expected loss from random tokens**

In [ ]:
# the expected loss of random tokens is log(vocab size) (or is it??)
expectedLoss = -torch.log(torch.tensor(1/tokenizer.vocab_size)) # math note: -log(1/V) == log(V)
print(f'Expected loss: {expectedLoss:.4f}')

In [ ]:
losses = np.zeros(10)
for i in range(len(losses)):

  # generate random tokens
  randtokens = torch.randint(tokenizer.vocab_size,(1,1024))

  # get the loss
  with torch.no_grad():
    outputs = model(randtokens,labels=randtokens)
  losses[i] = outputs.loss.item()

  print(f'Finished iteration {i+1} of {len(losses)}')

In [ ]:
# the text the model tried to predict :|
tokenizer.decode(randtokens[0])

In [ ]:
plt.figure(figsize=(8,3))

plt.plot(losses,'kh',markerfacecolor=[.9,.7,.7],markersize=12,label='Empirical')
plt.axhline(expectedLoss.item(),color=[.7,.7,.9],linestyle='--',label='Expected')

plt.gca().set(xlabel='Iteration',ylabel='Loss')

plt.tight_layout()
plt.savefig('ch4_proj19_part6.png')
plt.show()